# Start!

## Problem:
Mamy doczynienia z serią danych numerycznych wraz z problemem klasyfikacji binarnej.

Na podstawie opisów cech można wywnioskować, że są one w jakimś stopniu ze sobą powiązane - chociażby szerokość serca czy płuc z ich polem powierzchni.

### Moja początkowa intuicja:
1. Dokonać analizy PCA na cechach, by ograniczyć szum informacji
2. Zastosować klasyfikację knn z cross-examination na hiperparametrze k, by uniknąć under lub overfittingu

In [1]:
import pandas as pd
import torch
import numpy as np

ModuleNotFoundError: No module named 'pandas'

In [ ]:
from torch.utils.data import Dataset, DataLoader

class HeartParametersDataset(Dataset):
    """Dataset of heart parameters"""

    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

dataset = HeartParametersDataset("./task_data.csv")
print(len(dataset))
